# Descripcíon del proyecto


La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.


Tienes acceso a los datos de comportamiento de los suscriptores que ya se han cambiado a los planes nuevos (del proyecto del curso de Análisis estadístico de datos). Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.


Desarrolla un modelo con la mayor exactitud posible. En este proyecto, el umbral de exactitud es 0.75. Usa el dataset para comprobar la exactitud.

# Descripcíon de los datos

Cada observación en el dataset contiene información del comportamiento mensual sobre un usuario.

La información dada es la siguiente:

сalls — número de llamadas,

minutes — duración total de la llamada en minutos,

messages — número de mensajes de texto,

mb_used — Tráfico de Internet utilizado en MB,

is_ultra — plan para el mes actual (Ultra - 1, Smart - 0).

# Plan de acción 

Abre y examina el archivo de datos. Dirección al archivo:datasets/users_behavior.csv

Descarga el dataset

Segmenta los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.

Investiga la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.

Comprueba la calidad del modelo usando el conjunto de prueba.

Tarea adicional: haz una prueba de cordura al modelo. Estos datos son más complejos que los que habías usado antes así que no será una tarea fácil. Más adelante lo veremos con más detalle.

# Observación importante 

Para esta tarea de clasificación debes crear un modelo que escoja el plan correcto. Como ya hiciste el paso de procesar los datos, puedes lanzarte directo a crear el modelo.

# Importación de librerias

en esta seccion procedemos a importar todas las librerias que usaremos durante el proyecto

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Lectura de datos 

In [2]:
df = pd.read_csv("/datasets/users_behavior.csv")

In [3]:
#Observamos el dataset
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


# Segmentacion de datos

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.6, random_state=12345) 
#se segmenta el 60 % de los datos para hacer el conjunto de validación
features_valid, features_test, target_valid, target_test = train_test_split(features, target, test_size=0.5, random_state=12345)
#se segmenta el 50 % de los datos para hacer el conjunto de validación

# Investigacion de la calidad de diferentes modelos

## Árbol de decisión de clasificacion

In [5]:
best_model_tree = None
best_result = 0
best_depth = 0
for depth in range(1, 6): # selecciona el rango del hiperparámetro
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    # entrena el modelo en el conjunto de entrenamiento
    model.fit(features_train, target_train) 
    # entrena el modelo en el conjunto de entrenamiento
    predictions_valid = model.predict(features_valid)
    # obtén las predicciones del modelo en el conjunto de validación
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
        


print(f"El umbral de exactitud del mejor modelo en el conjunto de validación (max_depth = {best_depth}): {best_result}")

El umbral de exactitud del mejor modelo en el conjunto de validación (max_depth = 5): 0.8176726820161793


## Bosque aleatorio de clasificacion

In [6]:
best_model_forest = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model_forest = model
            best_result = result
            best_est = est
            best_depth = depth

           

print("umbral de exactitud del mejor modelo en el conjunto de validación", best_result, "n_estimators:", best_est, "best_depth:", depth)

umbral de exactitud del mejor modelo en el conjunto de validación 0.8817672682016179 n_estimators: 50 best_depth: 10


## Regresión logistica

In [7]:
model = LogisticRegression(random_state=54321, solver='liblinear') # inicializa el constructor de modelos
model.fit(features_train, target_train) 
# entrena el modelo en el conjunto de entrenamiento
predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación

result = accuracy_score(target_valid, predictions_valid)


print( "el umbral de exactitud del modelo de regresión logistica en el conjunto de validación:", result)

el umbral de exactitud del modelo de regresión logistica en el conjunto de validación: 0.7436216552582452


## Seleccion del modelo mas adecuado


Segun los analisis previos determinamos que el mejor accuracy lo tiene el modelo de bosque.
Por ende, vamos a seleccionar este modelo para proseguir.

# Comprobacion de calidad de modelo

In [8]:
predictions_test = best_model_forest.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print(result)

0.8070939639079029


# Prueba de cordura

In [9]:
predictions_cord = pd.Series(target_test.max(), index=target_test.index)

In [10]:
model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=10)
model.fit(features_valid, target_valid)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_cord)
print(result)

0.2899813316739266


In [11]:
predictions_cord_1 = pd.Series(target_test.min(), index=target_test.index)

model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=10)
model.fit(features_valid, target_valid)
predictions_test = model.predict(features_test)
result = accuracy_score(target_test, predictions_cord_1)
print(result)

0.7100186683260734


# Conclusion


Luego de probar 3 modelos diferentes, concluimos que el mejor modelo para esta situacion es el de ramdonforestclassifier, ya que en nuestro testeo tuvo un umbral de exactitud mas elevado que los demas.

Al comprobar la calidad del modelo seleccionado podemos observar que se mantiene por encima del margen que nos han otorgado que es 0.75, aunque disminuyo un poco en comparacion al resultado que nos dio antes.

Cuando realizamos la prueba de cordura introducimos datos en primer lugar solo 1 y luego solo 2.
y el modelo reacciona diferente en cada ocacion, dejandome entender que es coherente y funciona.